In [1]:
import pandas as pd 
from sodapy import Socrata
import os
import csv
import matplotlib.pyplot as plt


In [2]:
# url = https://data.austintexas.gov/resource/7d8e-dm7r.json
# Data Extraction:
client = Socrata("data.austintexas.gov", None)

results = client.get("7d8e-dm7r")

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [3]:
clean_results_df = results_df.copy()

In [4]:
clean_results_df = clean_results_df.rename(columns = {
    "trip_id": "Trip ID",
    "device_id": "Device ID",
    "modified_date": "Data Modified Date",
    "vehicle_type": "Vehicle Type",
    "trip_duration": "Trip Duration",
    "trip_distance": "Trip Distance",
    "start_time": "Trip Start Time",
    "end_time": "Trip End Time",
    "hour": "Hour",
    "day_of_week": "Day Of Week",
    "month": "Month",
    "year": "Year",
    "census_geoid_start": "GEOID Start",
    "census_geoid_end": "GEOID End",
    "council_district_start": "Start Council District",
    "council_district_end" : "Return Council District"
})

In [5]:
# Change the time and date format for columns - 'Data Modified Date', 'Trip Start time' and 'Trip End Time'
clean_results_df['Trip Start Time'] = pd.to_datetime(clean_results_df['Trip Start Time'])
clean_results_df['Trip End Time'] = pd.to_datetime(clean_results_df['Trip End Time'])
clean_results_df['Data Modified Date'] = pd.to_datetime(clean_results_df['Data Modified Date'])
clean_results_df

,Trip ID,Device ID,Data Modified Date,Start Council District,Return Council District,Vehicle Type,Trip Duration,Trip Distance,Trip Start Time,Trip End Time,Month,Hour,Day Of Week,Year,GEOID Start,GEOID End
0,6865731e-5e4b-4819-9435-4e62e09a652d,08c8c852-257c-49ad-a14f-727bfc79626d,2018-12-13 15:36:12,0,0,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1,08dc0d9c-735e-4006-b9af-60011509e469,255716b5-4a9f-4efc-9967-7a65b3521bac,2018-12-21 00:41:42,9,0,bicycle,11491603,1658,2018-11-30 11:00:00,2019-04-12 12:00:00,11,11,5,2018,NaN,NaN
2,92b80c20-89a4-4c7f-8b61-81a3570a218e,4d1e7bd0-75ce-401c-a802-8dd87122f802,2019-04-28 02:42:07,1,9,scooter,678,2785,2019-04-27 15:00:00,2019-04-27 15:15:00,4,15,6,2019,48453001100,48453001100
3,d9aef7c5-025d-4f2c-bafe-7801f89ef9dd,fdc38815-fe35-454b-bfb9-a63a2f2c205b,2019-04-28 02:42:07,9,5,scooter,713,2466,2019-04-27 15:00:00,2019-04-27 15:15:00,4,15,6,2019,48453001305,48453001901
4,baec3b6e-66f1-4043-bbd5-d54087f1c2de,007f1fed-0fe6-423f-8ca7-cd67834f20da,2019-04-28 02:42:07,9,9,scooter,469,1608,2019-04-27 15:15:00,2019-04-27 15:15:00,4,15,6,2019,48453001100,48453001305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1e0bd378-ea60-49e4-babe-8f27ee26e02a,394e7a8b-0d76-41b0-93d4-98ba9ecb13d1,2019-06-02 23:45:27,9,9,scooter,292,567,2019-06-01 22:45:00,2019-06-01 22:45:00,6,22,6,2019,48453001305,48453001401
996,6c29177d-2978-4d16-b013-3d6657e85500,f78faccd-8218-4bf0-92b0-5cc309fbebd6,2019-05-15 17:01:51,9,9,bicycle,138,450,2019-05-14 00:30:00,2019-05-14 00:30:00,5,0,2,2019,48453001100,48453001100
997,75a50782-933f-4eb2-a5b3-a8c570206503,6ba11e8b-42dc-4ba4-84eb-678155d278ea,2019-05-15 17:01:51,9,9,scooter,210,595,2019-05-14 02:15:00,2019-05-14 02:15:00,5,2,2,2019,48453000603,48453000603
998,0193c2cf-1c6c-4447-8f2f-f1cfccba3b92,0257125c-3d5b-4cab-baae-038ba9c17d8f,2019-05-15 17:36:46,9,9,scooter,134,209,2019-05-14 00:30:00,2019-05-14 00:30:00,5,0,2,2019,48453000604,48453000604


In [6]:
csvpath = os.path.join("ZIP_TRACT.csv")


zip_data = pd.read_csv(csvpath)
zip_data

             

zip_data[["zip", "GEOID End"]] = zip_data[["zip", "GEOID End"]].astype(str)

zip_data


zip_data_merge = pd.merge(clean_results_df,zip_data, on= "GEOID End", how ="right" )

zip_data_merge.dropna()

#merged_df = zip_data_merge

In [7]:
#merged_df

,Trip ID,Device ID,Data Modified Date,Start Council District,Return Council District,Vehicle Type,Trip Duration,Trip Distance,Trip Start Time,Trip End Time,...,Hour,Day Of Week,Year,GEOID Start,GEOID End,zip,res_ratio,bus_ratio,oth_ratio,tot_ratio
0,92b80c20-89a4-4c7f-8b61-81a3570a218e,4d1e7bd0-75ce-401c-a802-8dd87122f802,2019-04-28 02:42:07,1,9,scooter,678,2785,2019-04-27 15:00:00,2019-04-27 15:15:00,...,15,6,2019,48453001100,48453001100,78701,0.781628,0.804198,0.684404,0.784327
1,23787175-efa2-46db-9ba6-9d53f9eabd64,672b17a6-6916-4f4e-a4e4-8168d623f638,2019-04-28 02:42:07,9,9,scooter,644,1326,2019-04-27 15:15:00,2019-04-27 15:15:00,...,15,6,2019,48453001305,48453001100,78701,0.781628,0.804198,0.684404,0.784327
2,836370f2-60f0-47bf-a0fb-b26568e2745a,1ce0a7ba-b44c-4e37-9b8e-b92254f4de13,2019-04-28 02:42:07,9,9,scooter,307,973,2019-04-27 15:15:00,2019-04-27 15:15:00,...,15,6,2019,48453001200,48453001100,78701,0.781628,0.804198,0.684404,0.784327
3,6c32a1c0-b734-4fe8-934c-7eaef3e85791,10de4d7b-5e1e-48b5-887a-e88bf4761f99,2019-04-28 02:42:07,9,9,scooter,128,526,2019-04-27 15:15:00,2019-04-27 15:15:00,...,15,6,2019,48453001100,48453001100,78701,0.781628,0.804198,0.684404,0.784327
4,911faedf-58af-43de-923d-a2368d1270d2,c0a5c680-c577-43c8-a536-887c571d06f4,2019-04-28 02:42:07,9,9,scooter,621,1967,2019-04-27 15:00:00,2019-04-27 15:15:00,...,15,6,2019,48453001401,48453001100,78701,0.781628,0.804198,0.684404,0.784327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172893,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaT,...,NaN,NaN,NaN,NaN,2198000100,99927,0.000000,0.000000,1.000000,1.000000
172894,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaT,...,NaN,NaN,NaN,NaN,2198000200,99921,0.000000,0.000000,1.000000,1.000000
172895,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaT,...,NaN,NaN,NaN,NaN,2198000200,99925,0.000000,0.000000,1.000000,1.000000
172896,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaT,...,NaN,NaN,NaN,NaN,2198000300,99923,0.000000,0.000000,1.000000,1.000000


In [48]:
csvpath = os.path.join("ZIP_TRACT.csv")


zip_data = pd.read_csv(csvpath)
zip_data

             

zip_data[["zip", "GEOID End"]] = zip_data[["zip", "GEOID End"]].astype(str)

zip_data


zip_data_merge = pd.merge(clean_results_df,zip_data, right_on= "GEOID End", left_on = "GEOID End",
                          how ="right" )

zip_data_merge.drop(columns=['res_ratio', 'bus_ratio', 'oth_ratio','tot_ratio']).dropna()

,Trip ID,Device ID,Data Modified Date,Start Council District,Return Council District,Vehicle Type,Trip Duration,Trip Distance,Trip Start Time,Trip End Time,Month,Hour,Day Of Week,Year,GEOID Start,GEOID End,zip
0,92b80c20-89a4-4c7f-8b61-81a3570a218e,4d1e7bd0-75ce-401c-a802-8dd87122f802,2019-04-28 02:42:07,1,9,scooter,678,2785,2019-04-27 15:00:00,2019-04-27 15:15:00,4,15,6,2019,48453001100,48453001100,78701
1,23787175-efa2-46db-9ba6-9d53f9eabd64,672b17a6-6916-4f4e-a4e4-8168d623f638,2019-04-28 02:42:07,9,9,scooter,644,1326,2019-04-27 15:15:00,2019-04-27 15:15:00,4,15,6,2019,48453001305,48453001100,78701
2,836370f2-60f0-47bf-a0fb-b26568e2745a,1ce0a7ba-b44c-4e37-9b8e-b92254f4de13,2019-04-28 02:42:07,9,9,scooter,307,973,2019-04-27 15:15:00,2019-04-27 15:15:00,4,15,6,2019,48453001200,48453001100,78701
3,6c32a1c0-b734-4fe8-934c-7eaef3e85791,10de4d7b-5e1e-48b5-887a-e88bf4761f99,2019-04-28 02:42:07,9,9,scooter,128,526,2019-04-27 15:15:00,2019-04-27 15:15:00,4,15,6,2019,48453001100,48453001100,78701
4,911faedf-58af-43de-923d-a2368d1270d2,c0a5c680-c577-43c8-a536-887c571d06f4,2019-04-28 02:42:07,9,9,scooter,621,1967,2019-04-27 15:00:00,2019-04-27 15:15:00,4,15,6,2019,48453001401,48453001100,78701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2789,74f81ec4-b4ff-41c6-80e4-6b4cfac284c5,d4427ac5-8850-4a77-b054-571ff3237e30,2019-04-28 03:00:18,3,3,scooter,1071,2106,2019-04-27 15:30:00,2019-04-27 15:45:00,4,15,6,2019,48453001308,48453002003,78745
2790,747a0163-46fa-46a0-82ea-0edc4700d0e2,6403f170-3c08-4ac2-996a-d9c9efd611af,2019-04-28 03:00:23,9,10,scooter,2588,9318,2019-04-27 15:00:00,2019-04-27 15:30:00,4,15,6,2019,48453001100,48453001606,78703
2791,e032e321-00a8-427b-a4a5-33123c59d031,6cb6469d-c159-4f16-861c-f1d829b53d99,2019-06-05 12:45:15,7,7,scooter,672,3115,2019-06-04 23:45:00,2019-06-05 00:00:00,6,23,2,2019,48453000203,48453001505,78752
2792,e032e321-00a8-427b-a4a5-33123c59d031,6cb6469d-c159-4f16-861c-f1d829b53d99,2019-06-05 12:45:15,7,7,scooter,672,3115,2019-06-04 23:45:00,2019-06-05 00:00:00,6,23,2,2019,48453000203,48453001505,78756


In [14]:
zip_data

,zip,GEOID End,res_ratio,bus_ratio,oth_ratio,tot_ratio
0,501,36103158607,0.000000,1.000000,0.000000,1.000000
1,601,72113071700,0.188722,0.209068,0.125000,0.188301
2,601,72001956800,0.017797,0.025189,0.083333,0.020032
3,601,72001956300,0.004934,0.005038,0.005952,0.004968
4,601,72001956600,0.166520,0.347607,0.345238,0.182853
...,...,...,...,...,...,...
170188,99925,2198000200,0.000000,0.000000,1.000000,1.000000
170189,99926,2198940100,0.000000,0.000000,1.000000,1.000000
170190,99927,2198000100,0.000000,0.000000,1.000000,1.000000
170191,99928,2130000100,0.000000,0.000000,1.000000,1.000000


In [10]:
other_merge =  pd.merge(clean_results_df, zip_data,  on= "GEOID End", how ="right" )
other_merge.drop(columns=['res_ratio', 'bus_ratio', 'oth_ratio','tot_ratio']).dropna()

,Trip ID,Device ID,Data Modified Date,Start Council District,Return Council District,Vehicle Type,Trip Duration,Trip Distance,Trip Start Time,Trip End Time,Month,Hour,Day Of Week,Year,GEOID Start,GEOID End,zip
0,92b80c20-89a4-4c7f-8b61-81a3570a218e,4d1e7bd0-75ce-401c-a802-8dd87122f802,2019-04-28 02:42:07,1,9,scooter,678,2785,2019-04-27 15:00:00,2019-04-27 15:15:00,4,15,6,2019,48453001100,48453001100,78701
1,23787175-efa2-46db-9ba6-9d53f9eabd64,672b17a6-6916-4f4e-a4e4-8168d623f638,2019-04-28 02:42:07,9,9,scooter,644,1326,2019-04-27 15:15:00,2019-04-27 15:15:00,4,15,6,2019,48453001305,48453001100,78701
2,836370f2-60f0-47bf-a0fb-b26568e2745a,1ce0a7ba-b44c-4e37-9b8e-b92254f4de13,2019-04-28 02:42:07,9,9,scooter,307,973,2019-04-27 15:15:00,2019-04-27 15:15:00,4,15,6,2019,48453001200,48453001100,78701
3,6c32a1c0-b734-4fe8-934c-7eaef3e85791,10de4d7b-5e1e-48b5-887a-e88bf4761f99,2019-04-28 02:42:07,9,9,scooter,128,526,2019-04-27 15:15:00,2019-04-27 15:15:00,4,15,6,2019,48453001100,48453001100,78701
4,911faedf-58af-43de-923d-a2368d1270d2,c0a5c680-c577-43c8-a536-887c571d06f4,2019-04-28 02:42:07,9,9,scooter,621,1967,2019-04-27 15:00:00,2019-04-27 15:15:00,4,15,6,2019,48453001401,48453001100,78701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2789,74f81ec4-b4ff-41c6-80e4-6b4cfac284c5,d4427ac5-8850-4a77-b054-571ff3237e30,2019-04-28 03:00:18,3,3,scooter,1071,2106,2019-04-27 15:30:00,2019-04-27 15:45:00,4,15,6,2019,48453001308,48453002003,78745
2790,747a0163-46fa-46a0-82ea-0edc4700d0e2,6403f170-3c08-4ac2-996a-d9c9efd611af,2019-04-28 03:00:23,9,10,scooter,2588,9318,2019-04-27 15:00:00,2019-04-27 15:30:00,4,15,6,2019,48453001100,48453001606,78703
2791,e032e321-00a8-427b-a4a5-33123c59d031,6cb6469d-c159-4f16-861c-f1d829b53d99,2019-06-05 12:45:15,7,7,scooter,672,3115,2019-06-04 23:45:00,2019-06-05 00:00:00,6,23,2,2019,48453000203,48453001505,78752
2792,e032e321-00a8-427b-a4a5-33123c59d031,6cb6469d-c159-4f16-861c-f1d829b53d99,2019-06-05 12:45:15,7,7,scooter,672,3115,2019-06-04 23:45:00,2019-06-05 00:00:00,6,23,2,2019,48453000203,48453001505,78756


In [12]:
csv_file = merged_df.to_csv("GeoStartID.csv")

In [9]:
client = Socrata("data.austintexas.gov", None)
results = client.get("i26j-ai4z", limit=900000)
results_df = pd.DataFrame.from_records(results)


#results_df.sort_values("sr_type_code",ascending = False )

results_df.sr_type_code.unique() 

shared_mobility = results_df.loc[results_df['sr_type_code'] == "DOCKMOBI"]



shared_mobility 

,sr_number,sr_type_code,sr_type_desc,sr_department_desc,sr_method_received_desc,sr_status_desc,sr_status_date,sr_created_date,sr_updated_date,sr_location,...,sr_location_map_page,sr_location_map_tile,:@computed_region_8spj_utxs,:@computed_region_jcrc_4uuy,:@computed_region_q9nd_rr82,:@computed_region_m2th_e4b7,:@computed_region_rxpj_nzrk,:@computed_region_a3it_2a2z,:@computed_region_e9j2_6w3z,sr_closed_date
61,20-00015385,DOCKMOBI,Shared Micromobility,Transportation,Phone,Closed,2020-01-13T15:46:58.000,2020-01-13T14:36:08.000,2020-01-13T15:46:58.000,"2822 RIO GRANDE ST, AUSTIN, TX 78705",...,585A,MJ24,9,43,10,217,50,2860,57,2020-01-13T15:46:58.000
116,20-00015298,DOCKMOBI,Shared Micromobility,Transportation,Spot311 Interface,Open,2020-01-13T14:05:09.000,2020-01-13T14:05:09.000,2020-01-13T14:05:09.000,"54 RAINEY ST, AUSTIN, TX 78701",...,615B,MJ21,9,52,10,502,24,2856,60,NaN
167,20-00015160,DOCKMOBI,Shared Micromobility,Transportation,Phone,Closed,2020-01-13T15:51:41.000,2020-01-13T12:56:19.000,2020-01-13T15:51:41.000,"2810 SAN PEDRO ST, AUSTIN, TX 78705",...,585A,MJ24,9,43,10,217,50,2860,57,2020-01-13T15:51:41.000
180,20-00015131,DOCKMOBI,Shared Micromobility,Transportation,Spot311 Interface,Closed,2020-01-13T16:00:32.000,2020-01-13T12:43:49.000,2020-01-13T16:00:32.000,"2219 POST RD, AUSTIN, TX 78704",...,614R,MH20,9,9,10,217,75,2859,49,2020-01-13T16:00:32.000
239,20-00014967,DOCKMOBI,Shared Micromobility,Transportation,Spot311 Interface,Open,2020-01-13T11:04:41.000,2020-01-13T11:04:41.000,2020-01-13T11:04:41.000,"511 W 7TH ST, AUSTIN, TX 78701",...,585S,MJ22,9,52,10,217,24,2856,60,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259421,18-00220040,DOCKMOBI,Dockless Mobility,Transportation,Phone,Closed,2018-09-26T14:42:44.000,2018-07-10T09:05:35.000,2018-09-26T14:42:44.000,"122 FREDERICK ST, AUSTIN, TX 78704",...,614Z,MH19,3,9,3,217,75,2859,47,2018-09-26T14:42:44.000
261042,18-00215309,DOCKMOBI,Dockless Mobility,Transportation,Phone,Closed,2018-09-26T14:46:37.000,2018-07-06T11:42:03.000,2018-09-26T14:46:37.000,"901 CHICON ST, AUSTIN, TX 78702",...,585Y,MK22,1,7,8,309,1,2857,59,2018-09-26T14:46:37.000
261328,18-00214399,DOCKMOBI,Dockless Mobility,Transportation,Phone,Closed,2018-09-26T14:50:43.000,2018-07-05T18:11:22.000,2018-09-26T14:50:43.000,"1609 ENFIELD RD, AUSTIN, TX 78703",...,584M,MH23,9,26,10,126,69,2858,72,2018-09-26T14:50:43.000
262028,18-00211272,DOCKMOBI,Dockless Mobility,Transportation,Phone,Closed,2019-01-09T10:09:41.000,2018-07-04T11:21:35.000,2019-01-09T10:09:41.000,"1172 WALLER ST, AUSTIN, TX 78702",...,585U,MK22,1,7,8,309,1,2857,56,2019-01-09T10:09:41.000


In [ ]:
geo_end_merge = pd.merge(complaint_311, zip_data,  on= "zip", how ="right" )

In [64]:
complaint_311 = pd.DataFrame({
    "Complaint Number": shared_mobility["sr_number"],
    "Complaint Type": shared_mobility["sr_type_desc"],
    "Complaint Date": shared_mobility["sr_created_date"],
    "complaint_location": shared_mobility["sr_location"],
    "zip": shared_mobility["sr_location_zip_code"],
    "complaint_latitude": shared_mobility["sr_location_lat"],
    "complaint_longitudge": shared_mobility["sr_location_long"]
                                           })

complaint_311.zip.unique()

array(['78705', '78701', '78704', '78702', '78751', '78758', '78722',
       '78745', '78754', '78746', '78741', '78756', '78703', '78731',
       '78664', '78660', nan, '78727', '78752', '78748', '78712', '78744',
       '78757', '78723', '78747', '78681', '78753', '78721', '78617',
       '78736', '78750', '78725', '78759', '78717', '78749', '78613',
       '78653', '78724', '78645', '78728', '78735', '78669', '78739',
       '78726', '78719', '78729'], dtype=object)

In [51]:
csvpath = os.path.join("ZIP_TRACT.csv") 


zip_data_start = pd.read_csv(csvpath)


#zip_data_start_df = pd.DataFrame(zip_data_start)
zip_data_start[["zip", "GEOID Start"]] = zip_data_start[["zip", "GEOID Start"]].astype(str)
             



zip_data_merge_start = pd.merge(clean_results_df,zip_data_start, on= "GEOID Start", how ="inner" )

zip_data_merge_start

#startGEO

#clean_results_df.join(zip_data_start_df, on ="GEOID Start")

zip_data_start =  pd.merge(clean_results_df,zip_data, left_on= "GEOID Start", right_on = "GEOID End", how ='left')

KeyError: "['GEOID Start'] not in index"

In [26]:
other_merge =  pd.merge(clean_results_df, zip_data,  on= "GEOID End", how ="right" )
other_merge.drop(columns=['res_ratio', 'bus_ratio', 'oth_ratio','tot_ratio']).dropna()

#GEOEND

,Trip ID,Device ID,Data Modified Date,Start Council District,Return Council District,Vehicle Type,Trip Duration,Trip Distance,Trip Start Time,Trip End Time,Month,Hour,Day Of Week,Year,GEOID Start,GEOID End,zip
0,92b80c20-89a4-4c7f-8b61-81a3570a218e,4d1e7bd0-75ce-401c-a802-8dd87122f802,2019-04-28 02:42:07,1,9,scooter,678,2785,2019-04-27 15:00:00,2019-04-27 15:15:00,4,15,6,2019,48453001100,48453001100,78701
1,23787175-efa2-46db-9ba6-9d53f9eabd64,672b17a6-6916-4f4e-a4e4-8168d623f638,2019-04-28 02:42:07,9,9,scooter,644,1326,2019-04-27 15:15:00,2019-04-27 15:15:00,4,15,6,2019,48453001305,48453001100,78701
2,836370f2-60f0-47bf-a0fb-b26568e2745a,1ce0a7ba-b44c-4e37-9b8e-b92254f4de13,2019-04-28 02:42:07,9,9,scooter,307,973,2019-04-27 15:15:00,2019-04-27 15:15:00,4,15,6,2019,48453001200,48453001100,78701
3,6c32a1c0-b734-4fe8-934c-7eaef3e85791,10de4d7b-5e1e-48b5-887a-e88bf4761f99,2019-04-28 02:42:07,9,9,scooter,128,526,2019-04-27 15:15:00,2019-04-27 15:15:00,4,15,6,2019,48453001100,48453001100,78701
4,911faedf-58af-43de-923d-a2368d1270d2,c0a5c680-c577-43c8-a536-887c571d06f4,2019-04-28 02:42:07,9,9,scooter,621,1967,2019-04-27 15:00:00,2019-04-27 15:15:00,4,15,6,2019,48453001401,48453001100,78701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2789,74f81ec4-b4ff-41c6-80e4-6b4cfac284c5,d4427ac5-8850-4a77-b054-571ff3237e30,2019-04-28 03:00:18,3,3,scooter,1071,2106,2019-04-27 15:30:00,2019-04-27 15:45:00,4,15,6,2019,48453001308,48453002003,78745
2790,747a0163-46fa-46a0-82ea-0edc4700d0e2,6403f170-3c08-4ac2-996a-d9c9efd611af,2019-04-28 03:00:23,9,10,scooter,2588,9318,2019-04-27 15:00:00,2019-04-27 15:30:00,4,15,6,2019,48453001100,48453001606,78703
2791,e032e321-00a8-427b-a4a5-33123c59d031,6cb6469d-c159-4f16-861c-f1d829b53d99,2019-06-05 12:45:15,7,7,scooter,672,3115,2019-06-04 23:45:00,2019-06-05 00:00:00,6,23,2,2019,48453000203,48453001505,78752
2792,e032e321-00a8-427b-a4a5-33123c59d031,6cb6469d-c159-4f16-861c-f1d829b53d99,2019-06-05 12:45:15,7,7,scooter,672,3115,2019-06-04 23:45:00,2019-06-05 00:00:00,6,23,2,2019,48453000203,48453001505,78756


In [ ]:
#scooter_df.complaint_zip.unique() 


scooter = complaint_311["zip"].value_counts()

scooter

scooter_df = pd.DataFrame(scooter)
scooter_df



In [ ]:
other_merge =  pd.merge(complaint_311, zip_data,  on= "zip", how ="right" )
other_merge.drop(columns=['res_ratio', 'bus_ratio', 'oth_ratio','tot_ratio']).dropna()

In [ ]:
other_merge =  pd.merge(clean_results_df, zip_data,  on= "GEOID End", how ="right" )
other_merge.drop(columns=['res_ratio', 'bus_ratio', 'oth_ratio','tot_ratio']).dropna()

In [ ]:
plt.plot

In [ ]:
new_merged = pd.merge(other_merge, complaint_311, on = "zip", how ="right")
new_data_frame = new_merged.drop(columns=['res_ratio', 'bus_ratio', 'oth_ratio','tot_ratio']).dropna()

new_data_frame.drop_duplicates()